# MATH 441 Discrete Optimization Problems

## September 20, 2022

* Dual Problem
* Weak Duality Theorem
* Strong Duality Theorem
* Complementary Slackness

In [1]:
import numpy as np
import scipy.linalg as la
from scipy.optimize import linprog

In [2]:
def tableau(c,A,b):
    m,n = A.shape
    I = np.eye(m)
    T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
    return T

def pivot(T,k,l):
    E = np.eye(T.shape[0])
    E[:,l] = -T[:,k]/T[l,k]
    E[l,l] = 1/T[l,k]
    return E@T

## Dual Problem 

Let $A$ be an $m \times n$ matrix, $\mathbf{b} \in \mathbb{R}^m$ and $\mathbf{c} \in \mathbb{R}^n$. Consider the linear optimization problem:

$$
\begin{array}{rc}
\text{maximize:} & \mathbf{c}^T \mathbf{x} \\
\text{subject to:} & A \mathbf{x} \leq \mathbf{b} \\
& \mathbf{x} \geq 0
\end{array}
$$

The **dual problem** is:

$$
\begin{array}{rc}
\text{minimize:} & \mathbf{b}^T \mathbf{y} \\
\text{subject to:} & A^T \mathbf{y} \geq \mathbf{c} \\
& \mathbf{y} \geq 0
\end{array}
$$

The original problem is called the **primal problem**.

Given $A$, $\mathbf{b}$ and $\mathbf{c}$, the SciPy command to solve the primal problem is
```
linprog(-c,A,b)
```
and the command to solve the dual problem is:
```
linprog(b,-A.T,-c)
```

**Example.** (Vanderbei: Exercise 2.9) Solve the following problem and its dual:

$$
A = \left[ \begin{array}{rrr} 0 & 2 & 3 \\ 1 & 1 & 2 \\ 1 & 2 & 3 \\  \end{array} \right]
\hspace{1in}
\mathbf{b} = \left[ \begin{array}{r} 5 \\ 4 \\ 7 \end{array} \right]
\hspace{1in}
\mathbf{c} = \left[ \begin{array}{r} 2 \\ 3 \\ 4 \end{array} \right]
$$

In [3]:
A = np.array([[0,2,3],[1,1,2],[1,2,3]])
b = np.array([5,4,7])
c = np.array([2,3,4])

In [4]:
T = tableau(c,A,b)
print(T)

[[0. 2. 3. 1. 0. 0. 5.]
 [1. 1. 2. 0. 1. 0. 4.]
 [1. 2. 3. 0. 0. 1. 7.]
 [2. 3. 4. 0. 0. 0. 0.]]


In [5]:
T1 = pivot(T,2,0)
print(T1.round(2))

[[ 0.    0.67  1.    0.33  0.    0.    1.67]
 [ 1.   -0.33  0.   -0.67  1.    0.    0.67]
 [ 1.    0.    0.   -1.    0.    1.    2.  ]
 [ 2.    0.33  0.   -1.33  0.    0.   -6.67]]


In [6]:
T2 = pivot(T1,0,1)
print(T2.round(2))

[[ 0.    0.67  1.    0.33  0.    0.    1.67]
 [ 1.   -0.33  0.   -0.67  1.    0.    0.67]
 [ 0.    0.33  0.   -0.33 -1.    1.    1.33]
 [ 0.    1.    0.    0.   -2.    0.   -8.  ]]


In [7]:
T3 = pivot(T2,1,0)
print(T3.round(2))

[[  0.    1.    1.5   0.5   0.    0.    2.5]
 [  1.    0.    0.5  -0.5   1.    0.    1.5]
 [  0.    0.   -0.5  -0.5  -1.    1.    0.5]
 [  0.    0.   -1.5  -0.5  -2.    0.  -10.5]]


In [9]:
linprog(-c,A,b)

     con: array([], dtype=float64)
     fun: -10.49999999999877
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-1.15463195e-14, -4.00568467e-13,  5.00000000e-01])
  status: 0
 success: True
       x: array([1.50000000e+00, 2.50000000e+00, 1.35714333e-12])

Now let's solve the dual problem. Since $\mathbf{c} \geq 0$ we have $-A^T \mathbf{y} \leq -\mathbf{c}$ where entries of $-\mathbf{c}$ are less than 0. We need to solve the auxiliary problem.

In [10]:
T = tableau([0,0,0,-1],np.column_stack([-A.T,-np.ones(3)]),-c)
print(T)

[[ 0. -1. -1. -1.  1.  0.  0. -2.]
 [-2. -1. -2. -1.  0.  1.  0. -3.]
 [-3. -2. -3. -1.  0.  0.  1. -4.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]]


In [11]:
T1 = pivot(T,3,2)
print(T1)

[[ 3.  1.  2.  0.  1.  0. -1.  2.]
 [ 1.  1.  1.  0.  0.  1. -1.  1.]
 [ 3.  2.  3.  1.  0.  0. -1.  4.]
 [ 3.  2.  3.  0.  0.  0. -1.  4.]]


In [12]:
T2 = pivot(T1,0,0)
print(T2.round(2))

[[ 1.    0.33  0.67  0.    0.33  0.   -0.33  0.67]
 [ 0.    0.67  0.33  0.   -0.33  1.   -0.67  0.33]
 [ 0.    1.    1.    1.   -1.    0.    0.    2.  ]
 [ 0.    1.    1.    0.   -1.    0.    0.    2.  ]]


In [13]:
T3 = pivot(T2,1,1)
print(T3.round(2))

[[ 1.  -0.   0.5  0.   0.5 -0.5  0.   0.5]
 [ 0.   1.   0.5  0.  -0.5  1.5 -1.   0.5]
 [ 0.   0.   0.5  1.  -0.5 -1.5  1.   1.5]
 [ 0.   0.   0.5  0.  -0.5 -1.5  1.   1.5]]


In [14]:
T4 = pivot(T3,6,2)
print(T4.round(2))

[[ 1.  -0.   0.5 -0.   0.5 -0.5  0.   0.5]
 [ 0.   1.   1.   1.  -1.   0.   0.   2. ]
 [ 0.   0.   0.5  1.  -0.5 -1.5  1.   1.5]
 [ 0.   0.   0.  -1.   0.   0.   0.   0. ]]


In [15]:
T = tableau(-b,-A.T,-c)
print(T)

[[ 0. -1. -1.  1.  0.  0. -2.]
 [-2. -1. -2.  0.  1.  0. -3.]
 [-3. -2. -3.  0.  0.  1. -4.]
 [-5. -4. -7.  0.  0.  0.  0.]]


In [16]:
T1 = pivot(T,1,0)
print(T1)

[[ 0.  1.  1. -1.  0.  0.  2.]
 [-2.  0. -1. -1.  1.  0. -1.]
 [-3.  0. -1. -2.  0.  1.  0.]
 [-5.  0. -3. -4.  0.  0.  8.]]


In [17]:
T2 = pivot(T1,0,1)
print(T2.round(2))

[[ 0.   1.   1.  -1.   0.   0.   2. ]
 [ 1.   0.   0.5  0.5 -0.5  0.   0.5]
 [ 0.   0.   0.5 -0.5 -1.5  1.   1.5]
 [ 0.   0.  -0.5 -1.5 -2.5  0.  10.5]]


In [18]:
linprog(b,-A.T,-c)

     con: array([], dtype=float64)
     fun: 10.500000000002691
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([5.16919840e-13, 7.60724816e-13, 1.50000000e+00])
  status: 0
 success: True
       x: array([5.00000000e-01, 2.00000000e+00, 2.67783836e-14])

## Weak Duality Theorem

**Theorem.** (*Weak Duality Theorem*) If $\mathbf{x} \in \mathbb{R^n}$ is feasible for the primal and $\mathbf{y} \in \mathbb{R}^m$ is feasible for the dual then $\mathbf{c}^T \mathbf{x} \leq \mathbf{b}^T \mathbf{y}$.

*Proof.* Suppose $\mathbf{x}$ and $\mathbf{y}$ are feasible solutions of the primal and dual problems respectively. Then

$$
\mathbf{c}^T \mathbf{x}  = \mathbf{x}^T \mathbf{c}
\leq \mathbf{x}^T A^T \mathbf{y}
= \mathbf{y}^T A \mathbf{x}
\leq \mathbf{y}^T \mathbf{b}
$$

**Corollary.** If the primal problem is unbounded then the dual problem is infeasible.

*Proof.* Suppose $\mathbf{y}$ is a feasible solution of the dual problem. The weak duality theorem states that $\mathbf{c}^T \mathbf{x} \leq \mathbf{b}^T \mathbf{y}$ for all feasible solutions $\mathbf{x}$ of the primal problem. Therefore a feasible solution of the dual problem implies that the primal problem is bounded. Therefore if the primal problem is unbounded then the dual problem must be infeasible.

**Note.** If a problem is infeasible then the dual problem is either unbounded **or** infeasible. But if we can show that a problem is feasible and the dual is infeasible then the problem is unbounded.

**Example.** (Vanderbei: Reformulation of Exercise 2.3) Show that the following problem is unbounded

$$
A = \left[ \begin{array}{rrr} -1 & -1 & -1 \\ 2 & -1 & 1 \end{array} \right]
\hspace{1in}
\mathbf{b} = \left[ \begin{array}{r} -2 \\ 1 \end{array} \right]
\hspace{1in}
\mathbf{c} = \left[ \begin{array}{r} -2 \\ 6 \\ 0 \end{array} \right]
$$

Setup the dual problem is standard form: maximize $-\mathbf{b}^T \mathbf{y}$ subject to $-A^T \mathbf{y} \leq -\mathbf{c}$. The vector $-\mathbf{c}$ has negative entries therefore we setup the auxiliary problem for the dual:

In [19]:
A = np.array([[-1,-1,-1],[2,-1,1]])
b = np.array([-2,1])
c = np.array([-2,6,0])

In [20]:
np.column_stack([-A.T,-np.ones(3)])

array([[ 1., -2., -1.],
       [ 1.,  1., -1.],
       [ 1., -1., -1.]])

In [21]:
linprog([0.,0.,1.],np.column_stack([-A.T,-np.ones(3)]),-c)

     con: array([], dtype=float64)
     fun: 6.000000000004686
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([ 8.00000000e+00, -1.25410793e-11,  6.00000000e+00])
  status: 0
 success: True
       x: array([3.94837828e-14, 1.71879492e-11, 6.00000000e+00])

In [22]:
linprog(b,-A.T,-c)

     con: array([], dtype=float64)
     fun: 6.268577891267322
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([ 14.86158003, -12.59300214,   6.37671931])
  status: 2
 success: False
       x: array([0.10814142, 6.48486073])

The optimal value is not 0 therefore the dual problem is infeasible. We may conclude that the primal problem is either unbounded or infeasible. Setup the auxiliary problem for the primal problem:

In [23]:
linprog([0.,0.,0.,1.],np.column_stack([A,np.ones(2)]),b)

     con: array([], dtype=float64)
     fun: 1.3849037083675361e-12
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([0.80125785, 0.64660007])
  status: 0
 success: True
       x: array([5.68834256e-01, 1.50834609e+00, 7.24077508e-01, 1.38490371e-12])

The optimal value is 0 therefore the primal problem is feasible. Finally, we conclude the primal problem is unbounded.

In [24]:
linprog(-c,A,b)

     con: array([], dtype=float64)
     fun: -26732294971.685898
 message: 'The algorithm terminated successfully and determined that the problem is unbounded.'
     nit: 4
   slack: array([7.06745324e+09, 1.52659321e+09])
  status: 3
 success: False
       x: array([9.50155624e+08, 4.77210104e+09, 1.34519658e+09])

**Question.** Fix $m$ an $n$. If we randomly chose $A$ of size $m \times n$, $\mathbf{b} \in \mathbb{R}^m$ and $\mathbf{c} \in \mathbb{R}^n$, what is the probabilty that the problem is (1) feasible and unbounded (2) feasible and bound (3) unfeasible? 

## Strong Duality Theorem

**Theorem.** (*Strong Duality Theorem*) If $\mathbf{x} \in \mathbb{R^n}$ is an optimal feasible solution for the primal then there exists an optimal feasible solution $\mathbf{y} \in \mathbb{R}^m$ for the dual problem and $\mathbf{c}^T \mathbf{x} = \mathbf{b}^T \mathbf{y}$.

*Proof.* Start with the initial tableau

$$
T_0 = \begin{bmatrix} A & I & \mathbf{b} \\ \mathbf{c}^T & 0 & 0 \end{bmatrix}
$$

and apply the simplex method to reach the optimal tableau

$$
T_f = \begin{bmatrix} * & * & * \\ \mathbf{u}^T & \mathbf{v}^T & -\zeta \end{bmatrix}
$$

where all the entries of both $\mathbf{u}$ and $\mathbf{v}$ are less than or equal to 0. We obtain the optimal tableau by elementary row operations therefore the last line implies

$$
\mathbf{c}^T \mathbf{x} = \mathbf{u}^T \mathbf{x} + \mathbf{v}^T \mathbf{w} + \zeta
$$

where $\zeta$ is the optimal value $\mathbf{c}^T \mathbf{x}^*$. Rearrange using the equation $A \mathbf{x} + \mathbf{w} = \mathbf{b}$ to get

$$
\mathbf{c}^T \mathbf{x} = \mathbf{u}^T \mathbf{x} + \mathbf{v}^T (\mathbf{b} - A \mathbf{x}) + \zeta
= \mathbf{v}^T \mathbf{b} + \zeta + (\mathbf{u}^T - \mathbf{v}^T A) \mathbf{x}
$$

This equation holds for any $\mathbf{x}$ therefore $\mathbf{v}^T \mathbf{b} = -\zeta$ and

$$
\mathbf{u}^T - \mathbf{v}^T A = \mathbf{c}^T
\ \ \Rightarrow \ \ 
\mathbf{u} - A\mathbf{v} = \mathbf{c}
$$

Let $\mathbf{y}^* = -\mathbf{v}$. Then $\mathbf{y}^* \geq 0$ since all entries of $\mathbf{v}$ are less than or equal to 0. Compute

$$
A^T \mathbf{y}^* = -A^T\mathbf{v} = \mathbf{c} - \mathbf{u} \geq \mathbf{c}
$$

where the last inequality follows from the fact that all entries of $\mathbf{u}$ are less than or equal to 0. Therefore $\mathbf{y}^* = -\mathbf{v}$ is a feasible solution of the dual problem and

$$
\mathbf{b}^T \mathbf{y}^* = \zeta = \mathbf{c}^T \mathbf{x^*}
$$

and therefore it is optimal. $\square$

**Example.**

In [25]:
#A = np.random.randint(0,10,(2,3))
#c = np.random.randint(0,10,3)
#b = np.random.randint(0,10,2)

A = np.array([[5.,4.,1.],[1.,7.,6.]])
b = np.array([9.,7.])
c = np.array([8.,3.,8.])

T = tableau(c,A,b)
print(T)

[[5. 4. 1. 1. 0. 9.]
 [1. 7. 6. 0. 1. 7.]
 [8. 3. 8. 0. 0. 0.]]


In [26]:
Tdual = tableau(-b,-A.T,-c)
print(Tdual)

[[-5. -1.  1.  0.  0. -8.]
 [-4. -7.  0.  1.  0. -3.]
 [-1. -6.  0.  0.  1. -8.]
 [-9. -7.  0.  0.  0.  0.]]


In [26]:
T1 = pivot(T,2,1)
print(T1.round(2))

[[ 4.83  2.83  0.    1.   -0.17  7.83]
 [ 0.17  1.17  1.    0.    0.17  1.17]
 [ 6.67 -6.33  0.    0.   -1.33 -9.33]]


In [27]:
Tdual1 = pivot(Tdual,1,2)
print(Tdual1.round(2))

[[-4.83 -0.    1.    0.   -0.17 -6.67]
 [-2.83  0.    0.    1.   -1.17  6.33]
 [ 0.17  1.    0.    0.   -0.17  1.33]
 [-7.83  0.    0.    0.   -1.17  9.33]]


In [28]:
T2 = pivot(T1,0,0)
print(T2.round(2))

[[  1.     0.59   0.     0.21  -0.03   1.62]
 [  0.     1.07   1.    -0.03   0.17   0.9 ]
 [  0.   -10.24   0.    -1.38  -1.1  -20.14]]


In [29]:
Tdual2 = pivot(Tdual1,0,0)
print(Tdual2.round(2))

[[ 1.    0.   -0.21  0.    0.03  1.38]
 [ 0.    0.   -0.59  1.   -1.07 10.24]
 [ 0.    1.    0.03  0.   -0.17  1.1 ]
 [ 0.    0.   -1.62  0.   -0.9  20.14]]


## Complementary Slackness Theorem

**Theorem.** Let $\mathbf{x}$ be a feasible solution of the primal problem with corresponding slack vector $\mathbf{w}$. Let $\mathbf{y}$ be a feasible solution of the dual problem with corresponding slack vector $\mathbf{z}$. Then $\mathbf{x}$ and $\mathbf{y}$ are optimal if and only if $x_i z_i = 0$ for $i=0,\dots,n-1$, and $y_j w_j = 0$ for $j=0,\dots,m-1$.

**Note.** If $\mathbf{x}$ is an optimal solution of the primal problem such that the basic variables are nonzero, then we know which variables of an optimal solution of the dual problem are the basic variables and we can solve the dual system for the values of the basic variables.

In [30]:
A = np.array([[2.,1.,1.,3.],[1.,3.,1.,2.]])
b = np.array([5.,3.])
c = np.array([6.,8.,5.,9.])

primal = linprog(-c,A,b)
print(primal)

     con: array([], dtype=float64)
     fun: -17.000000000018527
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-1.57740487e-12, -5.69988501e-12])
  status: 0
 success: True
       x: array([2.00000000e+00, 2.63291606e-13, 1.00000000e+00, 2.26764410e-12])


In [31]:
dual = linprog(b,-A.T,-c)
print(dual)

     con: array([], dtype=float64)
     fun: 16.999999999991125
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-4.80770979e-12,  5.00000000e+00,  7.41628980e-13,  2.00000000e+00])
  status: 0
 success: True
       x: array([1., 4.])


In [32]:
M = np.hstack([-A.T,np.eye(4)])
y = la.solve(M[:,[0,1,3,5]],-c)
y

array([1., 4., 5., 2.])